## ORM

- 优点
	- 隐藏数据库实现，让业务代码访问对象而不是数据库表
	- 良好的数据库操作接口，简单、学习成本低
	- 动态数据表映射，在表结构甚至数据库改变时，减少代码修改量
	- 可以方便的引入缓存功能
- 缺点
	- 性能损失
	- 增加学习成本
	- 复杂查询实现不好

python ORM框架
- SQLAlchemy

### 主要内容
- SQLAlchemy ORM建立数据库连接以及创建表策略
- SQLAlchemy ORM插入和查询数据
- 高级话题

In [1]:
import sqlalchemy

print(sqlalchemy.__version__)

1.1.11


## Connecting

In [2]:
from sqlalchemy import create_engine

# ECHO显示原始语句
engine = create_engine('sqlite:///foo.db', echo=True)

## Declare a Mapping

In [3]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String
Base = declarative_base()


class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)

    def __repr__(self):
        return "<User(name='%s', fullname='%s', password='%s')>" % (self.name, self.fullname, self.password)

print(User.__table__)

users


## Crate a schema

In [6]:
Base.metadata.create_all(engine)

2017-06-21 19:27:43,870 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


2017-06-21 19:27:43,874 INFO sqlalchemy.engine.base.Engine ()


2017-06-21 19:27:43,878 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1


2017-06-21 19:27:43,888 INFO sqlalchemy.engine.base.Engine ()


2017-06-21 19:27:43,900 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")


2017-06-21 19:27:43,905 INFO sqlalchemy.engine.base.Engine ()


2017-06-21 19:27:43,913 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	password VARCHAR, 
	PRIMARY KEY (id)
)




2017-06-21 19:27:43,916 INFO sqlalchemy.engine.base.Engine ()


2017-06-21 19:27:43,935 INFO sqlalchemy.engine.base.Engine COMMIT


## Create an Instance of the Mapped Class

In [7]:
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')
ed_user.name
ed_user.password
ed_user.fullname
str(ed_user.id)

'None'

## Creating a Session

In [8]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session1 = Session()

## Adding and Updating Objects

In [9]:
session1.add(ed_user)

In [10]:
session1.flush()

2017-06-21 19:41:26,297 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


2017-06-21 19:41:26,301 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)


2017-06-21 19:41:26,307 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones', 'edspassword')


In [11]:
# 查询
our_user = session1.query(User).filter_by(name='ed').first()
our_user

2017-06-21 19:43:25,020 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?


2017-06-21 19:43:25,024 INFO sqlalchemy.engine.base.Engine ('ed', 1, 0)


<User(name='ed', fullname='Ed Jones', password='edspassword')>

In [12]:
# 执行commit后才写入数据库
session1.commit()

2017-06-21 19:46:14,383 INFO sqlalchemy.engine.base.Engine COMMIT


In [14]:
# 添加多个
session1.add_all([
    User(name='wendy', fullname='Wendy Williams', password='foobar'),
    User(name='mary', fullname='Mary Contrary', password='xxg527'),
    User(name='fred', fullname='Fred Flinstone', password='blah')])

In [15]:
session1.commit()

2017-06-21 19:50:08,202 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


2017-06-21 19:50:08,204 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)


2017-06-21 19:50:08,206 INFO sqlalchemy.engine.base.Engine ('wendy', 'Wendy Williams', 'foobar')


2017-06-21 19:50:08,212 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)


2017-06-21 19:50:08,215 INFO sqlalchemy.engine.base.Engine ('mary', 'Mary Contrary', 'xxg527')


2017-06-21 19:50:08,218 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)


2017-06-21 19:50:08,221 INFO sqlalchemy.engine.base.Engine ('fred', 'Fred Flinstone', 'blah')


2017-06-21 19:50:08,224 INFO sqlalchemy.engine.base.Engine COMMIT


In [16]:
ed_user.password = 'f8s7ccs'
session1.dirty

2017-06-21 19:51:38,162 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


2017-06-21 19:51:38,165 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname 
FROM users 
WHERE users.id = ?


2017-06-21 19:51:38,168 INFO sqlalchemy.engine.base.Engine (1,)


IdentitySet([<User(name='ed', fullname='Ed Jones', password='f8s7ccs')>])

In [18]:
session1.commit()

2017-06-21 19:52:47,978 INFO sqlalchemy.engine.base.Engine UPDATE users SET password=? WHERE users.id = ?


2017-06-21 19:52:47,982 INFO sqlalchemy.engine.base.Engine ('f8s7ccs', 1)


2017-06-21 19:52:47,989 INFO sqlalchemy.engine.base.Engine COMMIT


In [19]:
ed_user.id

2017-06-21 19:53:30,393 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


2017-06-21 19:53:30,397 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.id = ?


2017-06-21 19:53:30,400 INFO sqlalchemy.engine.base.Engine (1,)


1

## Rolling Back

In [21]:
ed_user.name = 'Edwardo'
fake_user = User(name='fakeuser', fullname='Invaild', password='12345')
session1.add(fake_user)

In [22]:
session1.query(User).filter(User.name.in_((['Edwardo', 'fakeuser']))).all()

2017-06-21 19:58:26,243 INFO sqlalchemy.engine.base.Engine UPDATE users SET name=? WHERE users.id = ?


2017-06-21 19:58:26,247 INFO sqlalchemy.engine.base.Engine ('Edwardo', 1)


2017-06-21 19:58:26,253 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)


2017-06-21 19:58:26,256 INFO sqlalchemy.engine.base.Engine ('fakeuser', 'Invaild', '12345')


2017-06-21 19:58:26,264 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?)


2017-06-21 19:58:26,266 INFO sqlalchemy.engine.base.Engine ('Edwardo', 'fakeuser')


[<User(name='Edwardo', fullname='Ed Jones', password='f8s7ccs')>,
 <User(name='fakeuser', fullname='Invaild', password='12345')>]

In [23]:
session1.rollback()

2017-06-21 19:58:56,240 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [26]:
print(ed_user.name)
fake_user in session1

ed


False

In [28]:
session1.query(User).filter(User.name.in_(['ed', 'fakeuser'])).all()

2017-06-21 20:01:37,667 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?)


2017-06-21 20:01:37,670 INFO sqlalchemy.engine.base.Engine ('ed', 'fakeuser')


[<User(name='ed', fullname='Ed Jones', password='f8s7ccs')>]